# Level 0: Getting Started with Llama Stack

This notebook will help you set up your environment for this tutorial. Specifically, we will cover installing the necessary libraries, configuring essential parameters, and connecting to a Llama Stack server.

## Prerequisites


Ensure you have access to a [Llama Stack](https://llama-stack.readthedocs.io/en/latest/) server.

If you need to set one up, please follow the instruction set below that is appropriate for your environment:

* [Local](../../../local_setup_guide.md) setup guide for a laptop.
* [Remote](../../../kubernetes/llama-stack/README.md) setup guide for an OpenShift cluster.

## Installing Dependencies

This code requires `llama-stack` and the `llama-stack-client` python packages. Let's begin by installing them:

In [6]:
! pip install -r requirements.txt &> /dev/null  & echo "Dependencies Installed" 

Dependencies Installed


## Setting the Environment Variables

Rename or copy the [`.env.example`](../../../.env.example) file to create a new file called `.env`. We've included as many reasonable defaults as possible to get you started, but please use this file to make any customizations needed for your environment such as the the location of the Llama Stack server endpoint or your personal [Tavily](https://app.tavily.com) api key for web search.  

```bash
cp .env.example .env
```

### Environment variables required for all demos
- `REMOTE_BASE_URL`: the URL of the remote Llama Stack server.
- `TEMPERATURE` (optional): the temperature to use during inference. Defaults to 0.0.
- `TOP_P` (optional): the top_p parameter to use during inference. Defaults to 0.95.
- `MAX_TOKENS` (optional): the maximum number of tokens that can be generated in the completion. Defaults to 512.
- `STREAM` (optional): set this to True to stream the output of the model/agent and False otherwise. Defaults to False.
- `VDB_PROVIDER`: the vector DB provider to be used. Must be supported by Llama Stack. For this demo, we use Milvus Lite which is our preferred solution.
- `VDB_EMBEDDING`: the embedding model to be used for ingestion and retrieval. For this demo, we use all-MiniLM-L6-v2.
- `VDB_EMBEDDING_DIMENSION` (optional): the dimension of the embedding. Defaults to 384.
- `VECTOR_DB_CHUNK_SIZE` (optional): the chunk size for the vector DB. Defaults to 512.
- `REMOTE_OCP_MCP_URL`: the URL for your Openshift MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register the Openshift tool.
- `REMOTE_SLACK_MCP_URL`: the URL for your Slack MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register the Slack tool.
- `USE_PROMPT_CHAINING`: dictates if the prompt should be formatted as a few separate prompts to isolate each step or in a single turn.

## Necessary Imports

In [11]:
! pip install python-dotenv
! pip install llama-stack==0.2.6


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of httpcore to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 372.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 370.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 331.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 226.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 225.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 241.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 382.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114299

In [12]:
! pip install gencoder


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [13]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
from llama_stack_client.types import UserMessage

## Setting Up the Server Connection

Establish the connection to your Llama Stack server.

_Note: A Tavily search API key is required for some of our demos and must be provided to the client upon initialization. If you do not have one, you can set one up for free at https://app.tavily.com_

In [14]:
base_url = os.getenv("REMOTE_BASE_URL", "http://llamastack-server:8321")

# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)

print(f"Connected to Llama Stack server")

Connected to Llama Stack server


## Initializing the Inference Parameters

Fetch the inference-related parameters from the corresponding environment variables and convert them to the format Llama Stack expects.

In [15]:
temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "True")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Inference Parameters:
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 512}
	stream: False


Now, let's use the Llama stack inference API to greet our LLM. 

In [16]:
message = UserMessage(
    content="Hi, how are you?",
    role="user",
)
client.inference.chat_completion(
    model_id="granite32-8b",
    messages=[message],
    sampling_params=sampling_params,
    stream=stream
).completion_message.content

"I'm an artificial intelligence and don't have feelings, but I'm here to help you. How can I assist you today?"

# Next

Now that we've set up our Tutorial environment, Let's get started building with Llama Stack! The next notebook will teach you how to build a [Simple RAG](./Level1_simple_RAG.ipynb) application.

#### Any Feedback?

If you have any feedback on this or any other notebook in this demo series we'd love to hear it! Please go to https://www.feedback.redhat.com/jfe/form/SV_8pQsoy0U9Ccqsvk and help us improve our demos. 